# Azure AI Agent service - Many agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.13.4 (main, Jun  3 2025, 15:34:24) [Clang 17.0.0 (clang-1700.0.13.3)]'

## Creaate connection to AI Foundry Project & instantiate a project client

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


## Instantiate AGENT 1 (Q&A)

In [5]:


agent1 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Q&A",
    instructions="You are an AI agent.",
)
print(f"Created agent, agent ID: {agent1.id}")

Created agent, agent ID: asst_FEGiG4z7CvqojLa0H8eNDF2x


## Instantiate AGENT 2 (Translator)

In [6]:


agent2 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Translator",
    instructions="You are an AI agent that translates English into Spanish.",
)
print(f"Created agent, agent ID: {agent2.id}")

Created agent, agent ID: asst_iXUNl1t2rYu8POTlK4odWb68


## Instantiate AGENT 3 (Emojis creator)

In [7]:


agent3 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Emojis",
    instructions="You are an AI agent that add emojis to a text",
)
print(f"Created agent, agent ID: {agent3.id}")


Created agent, agent ID: asst_8lhcGGxvECf2PBWb5R89p68K


### Show all agents

In [8]:
print(agent1)
print(agent2)
print(agent3)


{'id': 'asst_FEGiG4z7CvqojLa0H8eNDF2x', 'object': 'assistant', 'created_at': 1749398210, 'name': 'Q&A', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_iXUNl1t2rYu8POTlK4odWb68', 'object': 'assistant', 'created_at': 1749398210, 'name': 'Translator', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent that translates English into Spanish.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_8lhcGGxvECf2PBWb5R89p68K', 'object': 'assistant', 'created_at': 1749398211, 'name': 'Emojis', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent that add emojis to a text', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}


### Create a thread for AGENT 1

In [9]:
    # [START create_thread]
    thread = project_client.threads.create()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_x7Orpn1dWQu3waSpwPmzlkPy


### Create a message for AGENT 1

In [10]:

    # [START create_message]
    message = project_client.messages.create(thread_id=thread.id, role="user", content="Tell me about the 'Palacio de Bellas Artes' in Mexico City.")
    # [END create_message]
    print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_vZpzyKjFhknUbv1kjMPh3oWe


### Run AGENT 1

In [11]:
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent1.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")


Created run, run ID: run_Ju6hWREwW9Hg4x00I0nPBth5


### Check status of run for AGENT 1

In [12]:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


### Show usage (tokens)

In [13]:
run.usage

{'prompt_tokens': 41, 'completion_tokens': 402, 'total_tokens': 443, 'prompt_token_details': {'cached_tokens': 0}}

### Show me the stack of messages so far

In [14]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")





Message ID: msg_b54OgxkeKCJYP2if9vOzpwZ9, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'The **Palacio de Bellas Artes** (Palace of Fine Arts) is one of the most iconic and beautiful buildings in **Mexico City**. Here’s a concise overview:\n\n### Architectural Marvel\n\n- **Location**: At the western end of the historic center of Mexico City, near Alameda Central park.\n- **Construction**: Initiated in 1904 and completed in 1934, its construction was delayed by the Mexican Revolution and subsidence issues.\n- **Architectural Style**: A blend of Art Nouveau (exterior) and Art Deco (interior).\n- **Exterior**: Distinguished by its dazzling white Italian Carrara marble façade and a spectacular domed roof with orange and yellow colored glass.\n\n### Cultural Importance\n\n- **National Opera House**: Serves as Mexico’s foremost opera house.\n- **Performance Venue**: Home to the Ballet Folklórico de México, the National Symphony Orchestra, and other famed ensembles.\n

### Show last message of AGENT 1

In [15]:
messages = list(project_client.messages.list(thread_id=thread.id))
if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': 'The **Palacio de Bellas Artes** (Palace of Fine Arts) is one of the most iconic and beautiful buildings in **Mexico City**. Here’s a concise overview:\n\n### Architectural Marvel\n\n- **Location**: At the western end of the historic center of Mexico City, near Alameda Central park.\n- **Construction**: Initiated in 1904 and completed in 1934, its construction was delayed by the Mexican Revolution and subsidence issues.\n- **Architectural Style**: A blend of Art Nouveau (exterior) and Art Deco (interior).\n- **Exterior**: Distinguished by its dazzling white Italian Carrara marble façade and a spectacular domed roof with orange and yellow colored glass.\n\n### Cultural Importance\n\n- **National Opera House**: Serves as Mexico’s foremost opera house.\n- **Performance Venue**: Home to the Ballet Folklórico de México, the National Symphony Orchestra, and other famed ensembles.\n- **Murals**: Adorned with stunning murals by Diego Rivera, David A

### SHow bottom of stack of messages

In [16]:
if messages:
    last_message = messages[-1]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': "Tell me about the 'Palacio de Bellas Artes' in Mexico City.", 'annotations': []}}]


### Run AGENT 2

In [17]:
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent2.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")

Created run, run ID: run_djglRWuPJ6psN9iWjQacbe1n


### Check Status of the run for AGENT 2

In [18]:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [19]:
run.usage

{'prompt_tokens': 450, 'completion_tokens': 239, 'total_tokens': 689, 'prompt_token_details': {'cached_tokens': 0}}

### SHow the stack of messages so far

In [20]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")



Message ID: msg_HIdubEb1vJk0OOyH6KUWHDGd, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bellas Artes** es uno de los edificios más emblemáticos y hermosos de la Ciudad de México. Se encuentra en el centro histórico, junto a la Alameda Central. Su construcción comenzó en 1904 y concluyó en 1934, combinando estilos arquitectónicos art nouveau en el exterior y art déco en el interior.\n\nEste palacio es el principal recinto de espectáculos operísticos, teatrales y de danza del país. Es sede de la Orquesta Sinfónica Nacional, la Compañía Nacional de Ópera y el Ballet Folklórico de México. Además, alberga importantes murales de famosos artistas mexicanos como Diego Rivera, David Alfaro Siqueiros y Rufino Tamayo.\n\nEl Palacio de Bellas Artes también cuenta con museos dedicados al arte y la arquitectura. Uno de sus elementos más destacados es el telón de cristal, diseñado por Tiffany, que muestra una imagen del Valle de México con sus volcanes. En 198

### SHow last message of AGENT 2

In [21]:
messages = list(project_client.messages.list(thread_id=thread.id))
if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bellas Artes** es uno de los edificios más emblemáticos y hermosos de la Ciudad de México. Se encuentra en el centro histórico, junto a la Alameda Central. Su construcción comenzó en 1904 y concluyó en 1934, combinando estilos arquitectónicos art nouveau en el exterior y art déco en el interior.\n\nEste palacio es el principal recinto de espectáculos operísticos, teatrales y de danza del país. Es sede de la Orquesta Sinfónica Nacional, la Compañía Nacional de Ópera y el Ballet Folklórico de México. Además, alberga importantes murales de famosos artistas mexicanos como Diego Rivera, David Alfaro Siqueiros y Rufino Tamayo.\n\nEl Palacio de Bellas Artes también cuenta con museos dedicados al arte y la arquitectura. Uno de sus elementos más destacados es el telón de cristal, diseñado por Tiffany, que muestra una imagen del Valle de México con sus volcanes. En 1987, fue declarado Patrimonio de la Humanidad por la UNESCO, como par

### Run AGENT 3

In [22]:
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent3.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")

Created run, run ID: run_Hr7RXu3e8jeHYiVYicSX8cWL


### Check status of run of AGENT 3

In [23]:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [24]:
run.usage

{'prompt_tokens': 691, 'completion_tokens': 181, 'total_tokens': 872, 'prompt_token_details': {'cached_tokens': 0}}

### Show the Stack of messages so far

In [25]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")

Message ID: msg_RaBqDncekNdSbcNpMufK3pqW, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'Sure! Here’s a version enhanced with emojis:\n\nThe **Palacio de Bellas Artes** 🎭🏛️ is an iconic cultural landmark in **Mexico City** 🇲🇽. Located downtown near Alameda Central 🌳, it’s famous for its breathtaking architecture—a blend of **Art Nouveau** ✨ on the outside and **Art Deco** 🌀 on the inside.\n\nThe palace is a hub for the arts 🎨 and hosts performances of opera 🎶, ballet 🩰, and concerts. Inside, you’ll find stunning murals by famous Mexican artists such as Diego Rivera and Rufino Tamayo 🖼️. Don’t miss its magnificent Tiffany glass curtain featuring the Valley of Mexico and volcanoes 🌋.\n\nRecognized as a **UNESCO World Heritage Site** 🏵️, the Palacio de Bellas Artes is a must-see for anyone visiting Mexico City!', 'annotations': []}}]
Message ID: msg_HIdubEb1vJk0OOyH6KUWHDGd, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bell

### Print last message of AGENT 3

In [26]:
messages = list(project_client.messages.list(thread_id=thread.id))

last_message = messages[0]
print(f"Content: {last_message.content}")




Content: [{'type': 'text', 'text': {'value': 'Sure! Here’s a version enhanced with emojis:\n\nThe **Palacio de Bellas Artes** 🎭🏛️ is an iconic cultural landmark in **Mexico City** 🇲🇽. Located downtown near Alameda Central 🌳, it’s famous for its breathtaking architecture—a blend of **Art Nouveau** ✨ on the outside and **Art Deco** 🌀 on the inside.\n\nThe palace is a hub for the arts 🎨 and hosts performances of opera 🎶, ballet 🩰, and concerts. Inside, you’ll find stunning murals by famous Mexican artists such as Diego Rivera and Rufino Tamayo 🖼️. Don’t miss its magnificent Tiffany glass curtain featuring the Valley of Mexico and volcanoes 🌋.\n\nRecognized as a **UNESCO World Heritage Site** 🏵️, the Palacio de Bellas Artes is a must-see for anyone visiting Mexico City!', 'annotations': []}}]


## Post processing

In [33]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:


In [31]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")
        break

In [32]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")